<a href="https://colab.research.google.com/github/lucerogr/algorithmic-complexity/blob/main/TF_Copia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/lmcanavals/algorithmic_complexity.git

Cloning into 'algorithmic_complexity'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 104 (delta 50), reused 91 (delta 41), pack-reused 0
Receiving objects: 100% (104/104), 15.59 MiB | 28.81 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [2]:
from algorithmic_complexity.aclib import graphstuff as gs

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import math
import graphviz as gv

In [4]:
url="https://raw.githubusercontent.com/lmcanavals/algorithmic_complexity/main/data/poblaciones.csv"
poblacionesDF = pd.read_csv(url)
poblacionesDF=poblacionesDF.drop_duplicates(subset='CENTRO POBLADO')

In [5]:
def dist(cp1, cp2):
  la1, lo1 = float(cp1['LATITUD']), float(cp1['LONGITUD'])
  la2, lo2 = float(cp2['LATITUD']), float(cp2['LONGITUD'])
  
  lo1, la1, lo2, la2 = map(math.radians, [lo1, la1, lo2, la2])
  dlo = lo2 - lo1
  dla = la2 - la1
  a = math.sin(dla/2)**2 + math.cos(la1) * math.cos(la2) * math.sin(dlo/2)**2
  c = 2 * math.asin(math.sqrt(a))
  r = 6371

  return round(c * r, 2)

In [6]:
def dijkstra(G, s):
    unvisited = []
    visited = []
    total_weight = 0
    current_node = s
    neighbor = ''

    for node in G.nodes:
        if node == s:
            visited.append(s)
        else:
            unvisited.append(node)
    while unvisited:
        for i, neighbor in enumerate(unvisited):
            if i == 0:
                current_weight = G.edges[s, neighbor]['weight']
                current_node = neighbor
            elif G.edges[s, neighbor]['weight'] < current_weight:
                current_weight = G.edges[s, neighbor]['weight']
                current_node = neighbor
        total_weight += current_weight
        unvisited.remove(current_node)
        visited.append(current_node)
    
    #total_weight += G.edges[s, current_node]['weight']
    #visited.append(s)
    return visited, total_weight

In [7]:
nomprovincias = poblacionesDF['PROVINCIA'].unique()
provincias = dict()
distritos = dict()
import json 
nomdepartamentos = poblacionesDF['DEPARTAMENTO'].unique()
departamentos = dict()
G2=nx.Graph()
shortestPath = list()
for nomdepartamento in nomdepartamentos:
  departamentos[nomdepartamento] = poblacionesDF[poblacionesDF['DEPARTAMENTO'] == nomdepartamento]
  nomprovincias = departamentos[nomdepartamento]['PROVINCIA'].unique()
  for nom in nomprovincias:
    provincias[nom] = poblacionesDF[poblacionesDF['PROVINCIA'] == nom]
  
  for nomprovincia in nomprovincias:
    nomdistritos = provincias[nomprovincia]['DISTRITO'].unique()
  
    for nom in nomdistritos:
      distritos[nom] = provincias[nomprovincia][provincias[nomprovincia]['DISTRITO'] == nom]
      #print(nom, len(distritos[nom]))
      distrito = distritos[nom]
      G = nx.Graph()
      col = 'CENTRO POBLADO'
      for i, cp1 in distrito.iterrows():
        for j, cp2 in distrito.iterrows():
          if cp1[col] != cp2[col]:
            G.add_edge(cp1[col], cp2[col], weight=dist(cp1, cp2))
            #G2.add_node(list(G.nodes)[c])
          else:
            G.add_node(cp1[col])
      if len(G.nodes) > 0:
        path, weight = dijkstra(G, list(G.nodes)[0])
      else:
        path = list(G.nodes)
      responsePath = []
      for city in path:
        cp = distrito[distrito["CENTRO POBLADO"] == city]
        responsePath.append({"cp": city,
                            "lat": float(cp["LATITUD"]),
                            "lon": float(cp["LONGITUD"])})
      print(json.dumps(responsePath))
      #print(len(path))
      for i in range(len(path)):
        G2.add_node(path[i])
        shortestPath.append(responsePath[i])

[{"cp": "EL MUYO (C.P.L.)", "lat": -5.4225449999999995, "lon": -78.45299}, {"cp": "RESERVA UJUCANO", "lat": -5.427565, "lon": -78.444372}, {"cp": "TUTUMBEROS", "lat": -5.327317, "lon": -78.44471999999999}]
[{"cp": "ALENYA BAJO", "lat": -5.682439, "lon": -78.49522900000001}]
[{"cp": "MITOPAMPA", "lat": -5.93856, "lon": -77.96530200000001}]
[{"cp": "NUEVA ESPERANZA", "lat": -5.935482, "lon": -77.97675500000001}, {"cp": "VILLA ERNESTINA", "lat": -5.934992, "lon": -77.977918}, {"cp": "SE\u00d1OR DE GUALAMITA", "lat": -5.935217, "lon": -77.974682}, {"cp": "PEDRO RUIZ", "lat": -5.94069, "lon": -77.97635}, {"cp": "SANTA ROSA", "lat": -5.932149, "lon": -77.97210799999999}, {"cp": "PEDRO RUIZ GALLO", "lat": -5.941387000000001, "lon": -77.977098}, {"cp": "SAN JERONIMO", "lat": -5.95036, "lon": -77.97886}]
[{"cp": "ALCAPARROSA", "lat": -6.982367999999999, "lon": -77.93255500000001}]
[{"cp": "NARANJITOS", "lat": -5.814901, "lon": -78.277449}]
[{"cp": "AGUA AZUL", "lat": -5.969275, "lon": -78.61939

In [8]:
print(shortestPath)

[{'cp': 'EL MUYO (C.P.L.)', 'lat': -5.4225449999999995, 'lon': -78.45299}, {'cp': 'RESERVA UJUCANO', 'lat': -5.427565, 'lon': -78.444372}, {'cp': 'TUTUMBEROS', 'lat': -5.327317, 'lon': -78.44471999999999}, {'cp': 'ALENYA BAJO', 'lat': -5.682439, 'lon': -78.49522900000001}, {'cp': 'MITOPAMPA', 'lat': -5.93856, 'lon': -77.96530200000001}, {'cp': 'NUEVA ESPERANZA', 'lat': -5.935482, 'lon': -77.97675500000001}, {'cp': 'VILLA ERNESTINA', 'lat': -5.934992, 'lon': -77.977918}, {'cp': 'SEÑOR DE GUALAMITA', 'lat': -5.935217, 'lon': -77.974682}, {'cp': 'PEDRO RUIZ', 'lat': -5.94069, 'lon': -77.97635}, {'cp': 'SANTA ROSA', 'lat': -5.932149, 'lon': -77.97210799999999}, {'cp': 'PEDRO RUIZ GALLO', 'lat': -5.941387000000001, 'lon': -77.977098}, {'cp': 'SAN JERONIMO', 'lat': -5.95036, 'lon': -77.97886}, {'cp': 'ALCAPARROSA', 'lat': -6.982367999999999, 'lon': -77.93255500000001}, {'cp': 'NARANJITOS', 'lat': -5.814901, 'lon': -78.277449}, {'cp': 'AGUA AZUL', 'lat': -5.969275, 'lon': -78.619398}, {'cp': 